<a href="https://colab.research.google.com/github/akshit7093/CANTILEVER/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk scikit-learn pandas numpy

In [ ]:
!pip install joblib

**GPT**






In [ ]:
import nltk
import pandas as pd
import numpy as np
import joblib
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import save_model, load_model
from sklearn.metrics import accuracy_score

In [ ]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Load the IMDB dataset
def load_data():
    df = pd.read_csv('IMDB Dataset.csv')
    df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
    return df

df = load_data()

In [ ]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    return ' '.join([lemmatizer.lemmatize(token) for token in tokens if token.isalnum()])

df['processed_text'] = df['review'].apply(preprocess_text)

In [ ]:
print(df['processed_text'].head())

0    one of the other reviewer ha mentioned that af...
1    a wonderful little production br br the filmin...
2    i thought this wa a wonderful way to spend tim...
3    basically there a family where a little boy ja...
4    petter mattei love in the time of money is a v...
Name: processed_text, dtype: object


In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['processed_text']).toarray()
y = df['sentiment'].values

print("Shape of feature matrix:", X.shape)
print("Shape of target vector:", y.shape)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)


model = Sequential([
    Dense(64, activation='relu', input_shape=(5000,)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)


loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")


model.save('sentiment_model.h5')
# joblib.dump(tfidf, 'tfidf_vectorizer.joblib')


def predict_sentiment(review):
    processed = preprocess_text(review)
    features = tfidf.transform([processed]).toarray()
    prediction = model.predict(features)[0][0]
    return "Positive" if prediction > 0.5 else "Negative"

# Test the function
new_review = "This movie was fantastic! I loved every minute of it."
print(f"Sentiment: {predict_sentiment(new_review)}")

Shape of feature matrix: (50000, 5000)
Shape of target vector: (50000,)
Training set shape: (40000, 5000)
Testing set shape: (10000, 5000)
Epoch 1/10
1000/1000 [==============================] - 9s 8ms/step - loss: 0.3737 - accuracy: 0.8404 - val_loss: 0.2706 - val_accuracy: 0.8859
Epoch 2/10
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2472 - accuracy: 0.9054 - val_loss: 0.2761 - val_accuracy: 0.8838
Epoch 3/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2082 - accuracy: 0.9227 - val_loss: 0.2964 - val_accuracy: 0.8801
Epoch 4/10
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1722 - accuracy: 0.9384 - val_loss: 0.3321 - val_accuracy: 0.8775
Epoch 5/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.1410 - accuracy: 0.9508 - val_loss: 0.3486 - val_accuracy: 0.8780
Epoch 6/10
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1142 - accuracy: 0.9617 - val_loss: 0.3793 - val_accuracy: 0.87

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 152ms/step
Sentiment: Positive


In [ ]:
new_review = "actors were so over dramatic and over acting "
print(f"Sentiment: {predict_sentiment(new_review)}")

1/1 [==============================] - 0s 31ms/step
Sentiment: Negative
